<a href="https://colab.research.google.com/github/Mr-McGL/RExamsUtils/blob/dev%2Fmgarcia/pyexams/test/PyExamsUtilsTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyExamsUtils: Build and test

## Init

***Scripts and extensions***

In [ ]:
%%capture __script_and_ext__output__
!mkdir -p /content/scripts/
%pushd /content/scripts/
try:
  !wget -O importExt_script.py https://gist.githubusercontent.com/Mr-McGL/661ae7e50a1cfe7dfd5c0b23216bf0c3/raw/importExt_script.py

  %cd ..
  %run scripts/importExt_script skip_ext
  %run scripts/importExt_script pNpDir_ext
  %run scripts/importExt_script writefileE_ext
  %run scripts/importExt_script extended_output_ext
  #ToDo: use pydrive2
  %run scripts/importExt_script repo_ext

finally:
  %popd

In [ ]:
%%exout -hh 100px
__script_and_ext__output__()

## Build
To set up the build, make sure to configure the following parameters:

* `url`: The repository URL.
* `branch`: Specify the branch you want to build from.

In [ ]:
url = "https://github.com/Mr-McGL/RExamsUtils.git"
branch = "dev/mgarcia"

In [ ]:
%%exout -hh 400px -o __build_output__
%%pushAndPopDir -swd
#private repositories
#%%repo -t github -f reu  {url} {tokenFileID}
#%%pushAndPopDir -swd

import importlib

def prntStep(*args, **kwargs):
  print(f"\x1b[1m\x1b[36m", end='')
  print(*args, "\x1b[0m", **kwargs)

def installPKG(pkg):
  if importlib.util.find_spec(pkg) is None:
    !pip install {pkg}
  else:
    !pip install --upgrade {pkg}

################################
# Required Packages
prntStep("\nInstalling Required Packages")
installPKG("build")
installPKG("virtualenv")

##############################
# Clonning the Repository
# (for private repositories use %%repo)
prntStep("\nCloning the repository")
!git clone --recursive {url} reu

##############################
# Set Branch
prntStep("\nSetting the required branch")
%cd reu
!git checkout {branch}


##############################
# Build
prntStep("\nBuilding package")
%cd pyexams
#!python3 -m build
!python3 -m build

##############################
# Copying Package
prntStep("\nCopying package files")
!mkdir -p ../../pkgs
!cp -rf ./dist/* ../../pkgs

##############################
# Delete repository
# (for private repositories use %%repo)
prntStep("\nDeleting repository")
%cd ../..
!rm -r reu

In [ ]:
%%exout -hh 200px -cl
__build_output__

In [ ]:
%%skip {True}
%%pnpd -swd
#%%pushAndPopDir -swd
!rm -r reu

## Upload Binaries

Before uploading the package binaries, ensure that you've set the following parameters:

* `url`: The URL of the repository for binaries.
* `tokenFileID`: The Google Drive file ID where the your tokens are stored.
* `name`: Your user name.
* `email`: Your email address.

***How to create a token file:***

<details>

```py
%%resize_output -hh 60px
!apt-get install xattr
```

```py
%%pushAndPopDir -swd

tokenFileName = "_tokens_.json"
folderName =""

from google.colab import drive, userdata
from subprocess import getoutput
import json

drive.mount('gdrive')
repoURL = userdata.get('repoURL')

%pushd gdrive/MyDrive/{folderName}

with open(tokenFileName,"w") as f:
  f.write(json.dumps({
  "github": {
		  "user": userdata.get('user_name'),# #<---- Your user name
		  "email": userdata.get('user_email'), #<---- Your email
		  "token": userdata.get('token') #<---- Your token
	  }
  }))

%popd

drive.flush_and_unmount()
```

```py
#https://gist.github.com/korakot/9bec3d134a70aa9797e17bc0adeb6663
drive.mount('gdrive')
%pushd gdrive/MyDrive/{folderName}
file_ID = getoutput(f"xattr -p 'user.drive.id' {tokenFileName}")
%popd
drive.flush_and_unmount()
print(file_ID)
```

</details>

In [ ]:
# @title  { run: "auto", form-width: "50%", display-mode: "form" }
skip_upload = True # @param {type:"boolean"}

In [ ]:
%%skip {skip_upload}

#tokenFileID = "1cCDt2uhK4vnwpiStmyyjzci3u90JaIE3" #safe
#Stored as Google Colab secret
from google.colab import userdata
tokenFileID = userdata.get('tokenFileID')
email = userdata.get('user_email')
name = userdata.get('user_name')
url = "https://github.com/Mr-McGL/RExamsUtilsBinaries.git"

In [ ]:
%%skip {skip_upload}
!git config --global user.email {email}
!git config --global user.name {name}

In [ ]:
%%exout -hh 100px -o __upload_output__
%%skip {skip_upload}
%%pnpd -swd
%%repo -t github -f reu_bin  {url} {tokenFileID}
%%pnpd -swd

%cd reu_bin

!mkdir -p ./pkgs
!cp -rf ../pkgs/* ./pkgs
!git add *
!git commit -a -m "PKGs updated"
!git push

In [ ]:
__upload_output__

## Install

In [ ]:
%%exout -hh 400px -o __build_install__
%%pushAndPopDir -swd pkgs
import os
#files = [entry.name for entry in os.scandir('.') if entry.is_file() and entry.name.endswith(".whl")]
files = sorted([f for f in os.listdir('.') if f.endswith(".whl")])
#!pip install --force-reinstall {files[-1]}
print (f"pip install {files[-1]}")
!pip install {files[-1]}

In [ ]:
%%exout -hh 200px -cl
__build_install__

In [ ]:
%%skip True
%%exout -hh 200px
%%pushAndPopDir -swd pkgs

files = sorted([f for f in os.listdir('.') if f.endswith(".whl")])
!pip uninstall {files[-1]}

In [ ]:
import pyexams.fprnt as exfprint
exfprint.fprnt("Testing", fg="r")

import pyexams.ex
pyexams.ex.run("ls")

import pyexams
pyexams.types.Struct({1:2})

## Test

In [ ]:
%%exout -hh 100px -cl
!pip install nb-mypy
%load_ext nb_mypy
%nb_mypy -v
%nb_mypy Off

### Development

In [ ]:
# @title  { run: "auto", display-mode: "form" }
dev = False # @param {type:"boolean"}

In [ ]:
#Init
%%skip {not dev}
%%exout -hh 100px
%%pushAndPopDir -swd

url = "https://github.com/Mr-McGL/RExamsUtils.git"
branch = "dev/mgarcia"

import importlib

def prntStep(*args, **kwargs):
  print(f"\x1b[1m\x1b[36m", end='')
  print(*args, "\x1b[0m", **kwargs)

def installPKG(pkg):
  if importlib.util.find_spec(pkg) is None:
    !pip install {pkg}
  else:
    !pip install --upgrade {pkg}

prntStep("\nInstalling Required Packages")
installPKG("build")
installPKG("virtualenv")

prntStep("\nCloning the repository")
!git clone --recursive {url} reu

prntStep("\nSetting the required branch")
%cd reu
!git checkout {branch}

In [ ]:
#Build and install
%%skip {not dev}
%%exout -hh 100px
%%pushAndPopDir -swd reu/pyexams

import os

#prntStep("\nUninstalling previous version")
#if os.path.exists("./dist"):
#  %pushd "./dist"
#  files = sorted([f for f in os.listdir('.') if f.endswith(".whl")])
#  !pip uninstall -y {files[-1]}
#  %popd

prntStep("\nCleaning repo")
dzi = !find . -name *Zone.Identifier
dcch = !find . -name __pycache__
degg = !find . -name *.egg-info

for f in dzi + dcch + degg:
  !rm -r {f}

if os.path.exists("./build"):
  !rm -r ./build

if os.path.exists("./dist"):
  !rm -r ./dist


prntStep("\nBuilding repo")
!python3 -m build

prntStep("\nInstalling repo")
#!pip uninstall .
#%cd "./dist"
#files = sorted([f for f in os.listdir('.') if f.endswith(".whl")])
#!pip install {files[-1]}
#!pip install --force-reinstall {files[-1]}
#!pip install --force-reinstall .

In [ ]:
%%skip {not dev}
%%exout -hh 100px
%%pushAndPopDir -swd reu/pyexams/dist
prntStep("\nInstalling repo")
files = sorted([f for f in os.listdir('.') if f.endswith(".whl")])
#!yes | pip uninstall {files[-1]}
!pip install {files[-1]}

In [ ]:
import pyexams.ex

In [ ]:
%%skip {not dev}

import pyexams.fprnt as exfprint
exfprint.fprnt("Hola", fg="r")

import pyexams.ex
pyexams.ex.run("ls")

import pyexams
pyexams.types.Struct({1:2})

In [ ]:
%%skip {not dev}
%%exout -swd
prntStep("\nDeleting repository")
!rm -r reu

### ***fprnt*** - Formatted print tests

Los siguientes diccionarios incluyen caracteres especiales.

In [ ]:
from pyexams.fprnt import (format as frmt, fgColor as fgc,
                             bgColor as bgc, special as spcl)

print(f"{frmt.bold+frmt['italic']}Bold and italic{frmt.resetBold} - italic{frmt.reset} - default")
print(f"{frmt.b+frmt.i}Bold and italic{frmt.ri} - italic{frmt.rst} - default")

print("")
print(f"{fgc.cyan+bgc.magenta}Cyan and magenta{frmt.rst} | default")
print(f"{fgc.c+bgc.m}Cyan and magenta{fgc.default} | default and Magenta {bgc.d} default")

print(f"\n Unseen {spcl.lbeging}Line deleted")


In [ ]:
from pyexams.fprnt import fgvColor as fgvc, bgvColor as bgvc, fgRGB, bgRGB


print(f"{fgvc(1)+bgvc(50)}Test 1")
print(f"{fgRGB(200,0,0)+bgRGB(200,200,200)}Test 2")

print(f"{fgRGB(200,0,0)+bgRGB(200,200,200)}Test 2")



In [ ]:
from pyexams.fprnt import setFormat as sf

print(f"{sf(('fg',200,0,0), ('bg',200,200,200),'b' )}Test 1")
print(f"{sf(('fg',200), ('bg',1),'b' )}Test 2")
print(f"{sf(('fg','r'), ('bg','w'),'b' )}Test 3")
print(f"{sf('fgc','bgw','b' )}Test 4")
print(f"{sf('fgred','bgw','b' )}Test{sf('r')} 5")

La siguiente funcion permite seleccionar un formato.

In [ ]:
from pyexams.fprnt import fprnt, fprntNNL

fprnt("Test 1", fg = "r", bg = "w", frmt = "u")
fprnt("Test 2", fg = 2, bg = 3)
fprnt("Test 3", fg = (0,0,200), bg = (100,0,0))

fprntNNL("Test", fg = (0,100,200), bg = (100,0,0))
fprnt(" 4", fg = (200,50,0), bg = (0,200,0))

fprnt("Test", fg = (0,100,200), bg = (100,0,0), end=' ')
fprnt("5", fg = (200,50,0), bg = (0,200,0))

### Types

#### Check

In [ ]:
from pyexams.types import is_seq, is_seq_of_seq, is_seq_of_seq_or_bt

In [ ]:
from typing import Sequence
from pyexams.fprnt import setFormat as sf

def print_test_results(tresults: Sequence[bool], sep="\n", prefix=" * ", **kwargs):
  """
  Prints a string of "OK" or "KO" separated by the provided separator, " * " by default, for each
  element in the test results array, highlighting "OK" in green and "KO" in red.

  Args:
      tresults (Sequence[bool]): The test results array.
      sep (str): The separator between the "OK" or "KO" strings. Defaults to "\n".
      prefix (str): The prefix to prepend to each "OK" or "KO" string. Defaults to " * ".
      **kwargs: Keyword arguments to be passed to the `print()` function.
  """
  r = f'{sep}'.join(f"{sf('fgg','b')}{prefix}OK" if res else f"{sf('fgr','b')}{prefix}KO" for res in tresults)
  print(f"{r}{sf('fgg','r')}", sep=sep, **kwargs)

In [ ]:
%nb_mypy On

In [ ]:
%%exout -hh 100px -cl

print_test_results((
    is_seq([],str) == True,
    is_seq([1],str) == False,
    is_seq(["a"],str) == True,
    is_seq(["a"],int) == False,
    is_seq(["a", 1],(int,str)) == True,
    is_seq(["a", 1.0],(int,str)) == False,
    is_seq(["a", 1],(int,bool)) == False,
))

print("---")
print_test_results((
    is_seq_of_seq([('a','a'),('a',)], str) == True,
    is_seq_of_seq([('a','a'),('a')], str) == False,
    is_seq_of_seq([('a','a'),'a'], str) == False,
    is_seq_of_seq([('a','a'),['a']], str) == True,
    is_seq_of_seq([('a','a'),[1]], (str,int)) == True,
    is_seq_of_seq([('a','a'),[1]], str) == False,
))

print("---")
print_test_results((
    is_seq_of_seq_or_bt([('a','a'),('a',)], str) == True,
    is_seq_of_seq_or_bt([('a','a'),'a'], str) == True,
    is_seq_of_seq_or_bt(['a','a','a'], str) == True,
    is_seq_of_seq_or_bt(['a','a',[1]], str) == False,
    is_seq_of_seq_or_bt(['a','a',[1]], (str, int)) == True,
    ))


In [ ]:
%nb_mypy Off

### Install R and R/exams


In [ ]:
%%exout -hh 100px -cl
%%pnpd -swd scripts
!wget https://raw.githubusercontent.com/Mr-McGL/RExamsUtils/dev/mgarcia/nbscripts/download_rexams_utils.py
#-ne
%run download_rexams_utils.py -np -sld
%cd ../extensions
%load_ext sysrun_ext
%cd ..
%sysrun init_rexams

### ***R***

#### Run

In [ ]:
from pyexams.r import run as rrun
rrun("print(1)")

In [ ]:
!pip install

In [ ]:
#ToDo: Add tests

#### Output

In [ ]:
%nb_mypy On

In [ ]:
from pyexams.r import set_warnerr_cb, set_console_cb, run as rrun

In [ ]:
def error_test():
    rrun("""
      write("\nprints to stderr: ", stderr())
      for (i in 1:50) { cat("x", file = stderr())}
       write("", stderr())
    """)

In [ ]:
set_warnerr_cb(promt= "[R] <> ",
               promt_format = sf ('bi', ('fg', 0, 100 , 0)),
               msg_format = sf ('i', ('fg', 0, 200 , 0)) )
error_test()

In [ ]:
set_warnerr_cb(  msg_format = sf ('bi', ('fg', 200, 200 , 0)) )
error_test()

In [ ]:
set_warnerr_cb(promt= 'default', promt_format= 'default', msg_format= 'default')
error_test()

In [ ]:
set_warnerr_cb(default = True)
error_test()

In [ ]:
set_console_cb(default = True)
rrun("print('Standard output')")
set_console_cb(promt= "[R] <> ",  msg_format = sf ('bi', ('fg', 200, 200 , 0)))
rrun("print('Standard output')")

In [ ]:
%nb_mypy Off

### R/exams

#### _basics (only for develoment)

In [ ]:
%nb_mypy On

In [ ]:
from pyexams.exams._basics import __RawParam, __param

In [ ]:
from pyexams.fprnt import setFormat as sf

def ptest(t: bool, **kwargs):
  print(f"{sf('fgg','b')}OK{sf('r')}" if t else f"{sf('fgr','b')}KO{sf('r')}", **kwargs)

In [ ]:
ptest(isinstance("Hola", str))
ptest(isinstance(__RawParam("Hola"), str))
ptest(isinstance(__RawParam("Hola"), __RawParam))
ptest(not isinstance("Hola", __RawParam))

In [ ]:
%%exout -hh 100px -cl
__param.reset()
ptest(__param('', "first")=='\n\t"first"')
ptest(__param('a', __RawParam("first"))==',\n\ta = first')
ptest(__param("a", "a")==',\n\ta = "a"')
ptest(__param("a", 1)==',\n\ta = 1')
ptest(__param("a", True)==',\n\ta = TRUE')
ptest(__param("a", [1,2,3])==',\n\ta = c(1, 2, 3)')
ptest(__param("a", [True,False,True])==',\n\ta = c(TRUE, FALSE, TRUE)')
ptest(__param("a",("b","a"))==',\n\ta = c("b", "a")')
ptest(__param("a",("b",("a","c")))==',\n\ta = list("b", c("a", "c"))')
ptest(__param("a",(3,(2,1)))==',\n\ta = list(3, c(2, 1))')
ptest(__param("a",dict(b=(3,(2,1))))==',\n\ta = list(\n\t\tb = list(3, c(2, 1)))')


In [ ]:
%nb_mypy Off

#### Skeleton

In [ ]:
%nb_mypy On

In [ ]:
from pyexams.exams import skeleton

In [ ]:
%%pushAndPopDir -swd
%%exout -hh 200px -cl
skeleton(dir='exams/test1')
skeleton(dir='exams/test2', markup="latex")
skeleton(dir='exams/test3/md', addmarkup2path = False)
skeleton(dir='exams/test4', writer = "all", question_type = "all")
!ls -R exams | grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//--/g' -e 's/^/   /' -e 's/-/|/'
print("-----\n")
!find exams/test4 -type f -o -type d | sed -e 's/:$//' -e 's/[^-][^\/]*\//--/g' -e 's/^/   /' -e 's/-/|/'

!rm -r exams

In [ ]:
%nb_mypy Off

#### Render

In [ ]:
%nb_mypy On

In [ ]:
from typing import cast, Any
import random
import time
#from IPython.display import display
#from IPython.core.display import display
import ipywidgets as widgets
from glob import glob


from pyexams.fprnt import fprnt, setFormat as sf
from pyexams.r import set_warnerr_cb, set_console_cb
from pyexams.exams import skeleton, toHTML, toPDF, toNOPS, toMoodle, toPandoc, print_metadata

In [ ]:
from pathlib import Path
def file2str(fn):  return Path(fn).read_text()

def test_tile(title:str):
  fprnt("\n-------------------", fg="m", frmt = "b")
  print(title)
  fprnt("-------------------\n", fg="m", frmt = "b")


def run_test(func, *args, sep:bool = True, print_md: bool = True, **kwargs):
  md = func(*args, **kwargs) # <------ Funtion to test

  if print_md and isinstance(md, dict):
    print_metadata(cast(dict[Any, Any], md))
    print(f"seed.type: {md['seeds']['type']}\nseed.type: {md['seeds']['val']}")

  if sep: fprnt("\n-------------------\n", fg = "m", frmt = "b")

  return [md]

def run_toHtml(*args, **kwargs):
  return  run_test(toHTML, *args, **kwargs)

In [ ]:
%%exout -hh 300px -cl
%%pushAndPopDir -swd

# Set R console prompt format to make R exams messages visible
set_console_cb(promt= "| ",  msg_format = sf ('i', ('fg', 246)))

#Verbose test
#############
test_tile("Verbose test!")
for v in ('quiet', 'low', 'moderate', 'all'):
  print(f"Verbose level: {v}")
  run_toHtml(("swisscapital", ["anova", "deriv"], "boxhist", "countrycodes"), verbose = v, n = 2, gseed = 1, #toHtml params
                  print_md = False) #Test params

# Return metadata test
######################
test_tile("Return metadata test!")
mds = []
for rmd in (True, False):
  mds += run_toHtml("swisscapital", return_metadata = rmd, #toHtml params
                     sep = False, print_md = False)

print(f'None: {type(mds[-1])} - Metadata: {type(mds[-2])}')

# Name test #Encoding(qName) <- "UTF-8"
#######################################
test_tile("Name test!")
run_toHtml("swisscapital", name = "Test_", n = 3, #toHtml params
           sep = False, print_md = False) #Test params
!ls -l *.html

# Seed test
############
random.seed(int(time.time()))
test_tile("Seed test 1!")
for _ in range(2):
  run_toHtml(("swisscapital", ["anova", "deriv", "boxhist", "countrycodes"]), n = 2, gseed = 6) #toHtml params

test_tile("Seed test  2!")
mds = []
mds += run_toHtml(("swisscapital", ["anova", "deriv"], "boxhist", "countrycodes"), n = 2, seed = 5) #toHtml params
mds += run_toHtml(("swisscapital", ["anova", "deriv"], "boxhist", "countrycodes"), n = 2, seed = 5) #toHtml params
md = cast(dict[Any,Any], mds[-1])
mds += run_toHtml(("swisscapital", ["anova", "deriv"], "boxhist", "countrycodes"), n = 2, seed = md['seeds']['val']) #toHtml params

test_tile("Seed test  3!") # Local seeds not recomended in this scenario.
for _ in range(2):
  run_toHtml(("swisscapital", ["anova", "deriv", "boxhist", "countrycodes"]), n = 2, seed = 6) #toHtml params

# N-samples test
################
test_tile("N-samples test 1!")
for _ in range(2):
  run_toHtml(("swisscapital", ["anova", "deriv"], "boxhist", "countrycodes"), nsamp = 2, n = 2, gseed = 1) #toHtml params

test_tile("N-samples test 2!")
for _ in range(2):
  run_toHtml(("swisscapital", ["anova", "deriv"], "boxhist", "countrycodes"), nsamp = (1,2,1,1), n = 2, gseed = 1) #toHtml params

# Not recomended
test_tile("N-samples test 3!")
for _ in range(3):
  run_toHtml(("swisscapital", ["anova", "deriv"], "boxhist", "countrycodes"), nsamp = 2, n = 2, seed = 1) #toHtml params

#Partial solution
test_tile("N-samples test 4!")
for _ in range(3):
  run_toHtml((["swisscapital","swisscapital"], ["anova", "deriv"], ["boxhist","boxhist"]), nsamp = 2, n = 2, seed = 1) #toHtml params

# Question and solution test
############################
test_tile("Question and solution test 1!")
run_toHtml("swisscapital", name='T1qa_QS', gseed = 1, question = True, solution = True) #toHtml params
run_toHtml("swisscapital", name='T1qa_qS', gseed = 1, question = False, solution = True) #toHtml params
run_toHtml("swisscapital", name='T1qa_Qs', gseed = 1, question = True, solution = False) #toHtml params
display(widgets.HBox([widgets.HTML(file2str(fn)) for fn in glob('T1qa_*1.html')]))

test_tile("Question and solution test 2!")
run_toHtml("swisscapital", name='T2qa_', gseed = 1, question = "<h4>--Question--</<h4>", solution = "<h4>--Solution--</<h4>")
display(widgets.HBox([widgets.HTML(file2str('T2qa_1.html'))]))

# Folder test!
##############
test_tile("Folder test!")
run_toHtml("swisscapital",  odir="exams/result", edir="exams/markdown",  tdir="exams/tmp", sdir="exams/stmp", #toHtml params
           sep = False, print_md = False) #Test params
!ls -R exams | grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//--/g' -e 's/^/   /' -e 's/-/|/'
!rm -r "exams/result" "exams/tmp" "exams/stmp"

# exshuffle test!
##################
test_tile("exshuffle test!")
run_toHtml(("anova", "anova"),  gseed = 571940514); #toHtml params
run_toHtml(("anova", "anova"), n = 2, exshuffle=2, verbose='low', gseed = 571940514); #toHtml params

# RDS test!
###########
test_tile("RDS test!")
run_toHtml("anova", rds = True); #toHtml params
!ls *.rds
!rm *.rds


#ToDo: Additional Tests
#Img resolution

#toHtml("p", template = "template", svg = True, mathjax = True)
#toHtml("p", base64 = True)
#toHtml("p", base64 = ["png"])

set_console_cb(default = True)
!rm *.html

In [ ]:
toPandoc("anova",seed = 2, verbose='low')

In [ ]:
%%pushAndPopDir -swd
%nb_mypy Off
mds = [toHTML(["swisscapital", ["anova", "deriv"], "boxhist", "countrycodes"], **seed) \
       for seed in ({'seed':1}, {'gseed':1})]
!rm *.html
%nb_mypy On

In [ ]:
%%exout -hh 100px -cl
#%nb_mypy Off

for md in mds:
  fprnt('\n-------------------', fg='m')
  print_metadata(md)
  fprnt(md["seeds"],fg=242)

#%nb_mypy On

In [ ]:
%%pushAndPopDir -swd
import json

mdsfn = [f'metadata{i}.json' for i in range(len(mds))]
for fn, md in zip(mdsfn, mds):
  with open(fn, 'w') as f:
    json.dump(md, f, indent=2)

In [ ]:
%%pushAndPopDir -swd
import zipfile
with zipfile.ZipFile('metadata.zip', 'w') as zf:
  for fn in mdsfn: zf.write(fn)

In [ ]:
%%pushAndPopDir -swd
from google.colab import files
files.download("metadata.zip")
!rm metadata.zip metadata*.json